## Define Tetris Game

In [ ]:
import numpy as np
import random

class Tetris:
    
    UNDEFINED = -1
    
    TILES = [
        [
            [[0, 2]],  # Tile 0, orientation 0.
            [[0, 1], [0, 1]],  # Tile 0, orientation 1.
        ],
        [
            [[0, 1], [1, 2]],  # Tile 1, orientation 0.
            [[1, 2], [0, 1]],  # Tile 1, orientation 0.
        ],
        [
            [[0, 2], [1, 2]],  # Tile 2, orientation 0.
            [[0, 2], [0, 1]],  # Tile 2, orientation 1.
            [[0, 1], [0, 2]],  # Tile 2, orientation 2.
            [[1, 2], [0, 2]],  # Tile 2, orientation 3.
        ],
        [
            [[0, 2], [0, 2]],  # Tile 4, orientation 0.
        ],
    ]

    def __init__(self, rows, cols, max_tiles, random_seed):
        self.rows, self.cols = rows, cols
        self.max_tiles = max_tiles
        self.random_seed = random_seed
        
        self.restart()
        
    def restart(self):
        self.gameover = False
        self.tile_count = 0
        self.reward = 0
        self.board = np.full((self.rows, self.cols), Tetris.UNDEFINED)
        self.current_tile = Tetris.UNDEFINED
        self.tile_x = Tetris.UNDEFINED
        self.tile_y = Tetris.UNDEFINED
        self.tile_orientation = Tetris.UNDEFINED
        
        # Create predefined tile sequence, used if stochastic_prob=0
        rand_state = random.getstate()
        random.seed(self.random_seed)
        self.tile_sequence = [random.randint(0, len(Tetris.TILES) - 1) 
                              for x in range(self.max_tiles)]
        random.setstate(rand_state)
        
        self.next_tile()
        
    def next_tile(self):
        if self.tile_count < self.max_tiles:
            self.current_tile = self.tile_sequence[self.tile_count]
            self.tile_x = self.cols // 2
            self.tile_y = self.rows
            self.tile_orientation = 0
            self.tile_count += 1
        else:
            self.gameover = True
    
    def move_left(self):
        if self.tile_x - 1 >= 0:
            self.tile_x -= 1
            return True
        else:
            return False
    
    def move_right(self):
        tile_width = len(Tetris.TILES[self.current_tile][self.tile_orientation])
        if self.tile_x + 1 <= self.cols - tile_width:
            self.tile_x += 1
            return True
        else:
            return False
    
    def rotate(self):
        new_orientation = ((self.tile_orientation + 1) 
                           % len(Tetris.TILES[self.current_tile]))
        tile_width = len(Tetris.TILES[self.current_tile][new_orientation])
        if self.tile_x <= self.cols - tile_width:
            self.tile_orientation = new_orientation
            return True
        else:
            return False
        
    def drop(self):
        tile = Tetris.TILES[self.current_tile][self.tile_orientation]
        
        # Find first location where the piece collides with occupied locations.
        self.tile_y = 0
        for x in range(len(tile)):
            # Find first occupied location in this column            
            cury = -1
            for y in range(self.rows -1, -1, -1):
                if self.board[y, self.tile_x + x] > 0:
                    # Calculate the y position for this column if no other columns are taken into account
                    cury = y + 1 - tile[x][0]
                    break
            # Use the largest y position for all columns of the tile
            if self.tile_y < cury:
                self.tile_y = cury

        if self.tile_y + np.max(tile) > self.rows:
            self.gameover = True
            dreward = -100
        else:
            # Change board entries at the newly placed tile to occupied.
            for x in range(len(tile)):
                self.board[self.tile_y + tile[x][0]:self.tile_y + tile[x][1], 
                           x + self.tile_x] = 1

            # Remove full lines.
            removed_lines = 0
            for y in range(self.rows - 1, -1, -1):
                if np.sum(self.board[y, :]) == self.cols:
                    removed_lines += 1
                    for y1 in range(y, self.rows - 1):
                        self.board[y1, :] = self.board[y1 + 1, :]
                    self.board[self.rows - 1, :] = Tetris.UNDEFINED
            
            dreward = 10 ** (removed_lines - 1) if removed_lines > 0 else 0
            
            # Choose the next tile.
            self.next_tile()
        
        self.reward += dreward
        
        return dreward

## Adaptation to Q-Learning

In [ ]:
class QLTetris(Tetris):
    
    def __init__(self, rows, cols, max_tiles, random_seed):
        super().__init__(rows, cols, max_tiles, random_seed)
    
    def teleport(self, new_x, new_orientation):
        if 0 <= new_orientation < len(Tetris.TILES[self.current_tile]):  # Valid orientation.
            tile_width = len(Tetris.TILES[self.current_tile][new_orientation])
            if 0 <= new_x <= self.cols - tile_width:
                self.tile_x = new_x
                self.tile_orientation = new_orientation
                return True
        return False

In [ ]:
class QLAgent:
    
    def __init__(self, alpha, epsilon, games, **kwargs):
        self.alpha = alpha  # Alpha is the learning rate.
        self.epsilon = epsilon  # Probability to choose a random action in the epsilon-greedy policy.
        self.games = games
        self.game = 0
        self.tetris = QLTetris(**kwargs)
        
        self.rewards = np.zeros(games)

        self.state_size = (
            self.tetris.cols * self.tetris.rows  # Cells in board.
            + 1 + np.floor(np.log2(len(Tetris.TILES) - 1)).astype(int)  # Tiles
        )
        self.state_num = 2 ** self.state_size
        
        self.position_num = self.tetris.rows
        self.orientation_num = np.max([len(tile) for tile in Tetris.TILES])
        self.action_num = self.position_num * self.orientation_num

        self.Q_table = np.zeros((self.state_num, self.action_num))
        self.Q_target = self.Q_table
        
        self.update_state()

    def update_state(self):
        # Convert tile to binary list.
        tile = bin(self.tetris.current_tile)[2:]

        # Convert board to binary list.
        board = np.copy(self.tetris.board.reshape((-1,))).astype(int)
        board[board == Tetris.UNDEFINED] = 0
        
        self.state_binary = np.append(tile, board)
        self.state = int("".join(str(i) for i in self.state_binary), 2)

    def next_turn(self):
        if self.tetris.gameover:
            self.rewards[self.game] = self.tetris.reward
            if self.game % 100 == 0:
                av_reward = np.mean(self.rewards[self.game - 100:self.game])
                print(f"game {self.game}/{self.games} reward {av_reward}")
            
            self.game += 1
            if self.game < self.games:
                self.tetris.restart()
            else:
                np.savetxt('Q_table.txt', self.Q_table)
                return False  # Finish.
        else:
            old_state = self.state
            
            # Select action.
            if np.random.rand() < self.epsilon:
                action = np.random.randint(self.action_num)
            else:
                action = np.argmax(self.Q_table[old_state, :])
            
            # Extract rotation and movement from action parameter.
            new_x = action // self.position_num
            new_orientation = action % self.orientation_num

            # Execute action and drop tile.
            if self.tetris.teleport(new_x, new_orientation):
                reward = self.tetris.drop()
                    
                # Update the state.
                self.update_state()
                new_state = self.state
                
                # Update the Q-table using the old state and the reward.
                dQ = self.alpha * (reward
                                + np.max(self.Q_table[new_state, :])
                                - self.Q_table[old_state, action])
                            
                # Update the Q-table.
                self.Q_table[old_state, action] += dQ
            else:  # Penalty for illegal move.
                self.Q_table[old_state, action] += -50
            
        return True  # Continue.

In [ ]:
agent = QLAgent(
    alpha=0.2,
    epsilon=0, 
    games=10000, 
    rows=4,  # From here keyword arguments for Tetris.
    cols=4, 
    max_tiles=50, 
    random_seed=123456,
)

while agent.next_turn():
    pass

In [ ]:
import matplotlib.pyplot as plt

smoothed_rewards = np.convolve(agent.rewards, np.ones(100) / 100, mode='valid')

plt.plot(smoothed_rewards)
plt.ylabel('Reward')
plt.xlabel('Episode')
